In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import warnings
from datetime import datetime
import country_converter as coco
warnings.simplefilter('ignore')

In [2]:
df1 = pd.read_csv("FAOSTAT_data.csv")
df1

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,1371,"Almonds, in shell",2019,2019,ha,29203.00,A,Official figure,NaN
1,QCL,Crops and livestock products,4,Afghanistan,5412,Yield,1371,"Almonds, in shell",2019,2019,kg/ha,1308.30,A,Official figure,NaN
2,QCL,Crops and livestock products,4,Afghanistan,5510,Production,1371,"Almonds, in shell",2019,2019,t,38205.00,A,Official figure,NaN
3,QCL,Crops and livestock products,4,Afghanistan,5312,Area harvested,1371,"Almonds, in shell",2020,2020,ha,22134.00,A,Official figure,NaN
4,QCL,Crops and livestock products,4,Afghanistan,5412,Yield,1371,"Almonds, in shell",2020,2020,kg/ha,1775.90,A,Official figure,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224642,QCL,Crops and livestock products,716,Zimbabwe,5510,Production,22221.01,"Whole milk, evaporated",2019,2019,t,5010.21,I,Imputed value,NaN
224643,QCL,Crops and livestock products,716,Zimbabwe,5510,Production,22221.01,"Whole milk, evaporated",2020,2020,t,4795.70,I,Imputed value,NaN
224644,QCL,Crops and livestock products,716,Zimbabwe,5510,Production,22221.01,"Whole milk, evaporated",2021,2021,t,4921.05,I,Imputed value,NaN
224645,QCL,Crops and livestock products,716,Zimbabwe,5510,Production,22221.01,"Whole milk, evaporated",2022,2022,t,4966.37,I,Imputed value,NaN


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224647 entries, 0 to 224646
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Domain Code       224647 non-null  object 
 1   Domain            224647 non-null  object 
 2   Area Code (M49)   224647 non-null  int64  
 3   Area              224647 non-null  object 
 4   Element Code      224647 non-null  int64  
 5   Element           224647 non-null  object 
 6   Item Code (CPC)   224642 non-null  object 
 7   Item              224647 non-null  object 
 8   Year Code         224647 non-null  int64  
 9   Year              224647 non-null  int64  
 10  Unit              224645 non-null  object 
 11  Value             218130 non-null  float64
 12  Flag              224643 non-null  object 
 13  Flag Description  224634 non-null  object 
 14  Note              5098 non-null    object 
dtypes: float64(1), int64(4), object(10)
memory usage: 25.7+ MB


In [4]:
df1 = df1.drop(columns=['Domain Code','Domain','Area Code (M49)','Element Code','Item Code (CPC)','Note','Year Code','Flag','Flag Description'])
df1

,Area,Element,Item,Year,Unit,Value
0,Afghanistan,Area harvested,"Almonds, in shell",2019,ha,29203.00
1,Afghanistan,Yield,"Almonds, in shell",2019,kg/ha,1308.30
2,Afghanistan,Production,"Almonds, in shell",2019,t,38205.00
3,Afghanistan,Area harvested,"Almonds, in shell",2020,ha,22134.00
4,Afghanistan,Yield,"Almonds, in shell",2020,kg/ha,1775.90
...,...,...,...,...,...,...
224642,Zimbabwe,Production,"Whole milk, evaporated",2019,t,5010.21
224643,Zimbabwe,Production,"Whole milk, evaporated",2020,t,4795.70
224644,Zimbabwe,Production,"Whole milk, evaporated",2021,t,4921.05
224645,Zimbabwe,Production,"Whole milk, evaporated",2022,t,4966.37


In [5]:
missing_count = df1.isnull().sum()
missing_percent = (missing_count / len(df1)) * 100

summary = pd.concat([missing_count,missing_percent],axis=1, keys=['Count','Percent'])
print(summary[summary['Count'] > 0].sort_values('Count', ascending=False))

       Count   Percent
Value   6517  2.900996
Unit       2  0.000890


In [6]:
# Cleaning Area column
df1['Area']= df1['Area'].replace(
    {"China, Taiwan Province of": "Taiwan",
     "China, mainland":"China",
     "China, Hong Kong SAR":"Hong Kong",
     "T�rkiye":"Turkey",
     "C�te d'Ivoire":"Cote d'Ivoire",
     "Venezuela (Bolivarian Republic of)":"Venezuela",
     "Micronesia (Federated States of)":"Micronesia",
     "Bolivia (Plurinational State of)":"Bolivia",
     "Democratic People's Republic of Korea":"N.Korea",
     "Iran (Islamic Republic of)":"Iran",
     "Lao People's Democratic Republic":"Laos",
     "Micronesia (Federated States of)":"Micronesia",
     "Netherlands (Kingdom of the)":"Netherlands",
     "United Kingdom of Great Britain and Northern Ireland":"UK",
     "United States of America":"USA"}
)

In [7]:
cc = coco.CountryConverter()
df1['continent'] = df1['Area'].apply(lambda x: cc.convert(names=x, to='continent', not_found=None))

In [8]:
continent_list = list(df1['continent'].unique())
continent_list

['Asia', 'Europe', 'Africa', 'America', 'Oceania']

In [7]:
df1.to_csv('draft.csv')

In [8]:
# Extracting draft data to see how the data is after removing the unwanted cols and adding continent column for further analysis
df2 = pd.read_csv('draft.csv')
df2.drop(columns = 'Unnamed: 0', inplace=True)
df2

,Area,Element,Item,Year,Unit,Value
0,Afghanistan,Area harvested,"Almonds, in shell",2019,ha,29203.00
1,Afghanistan,Yield,"Almonds, in shell",2019,kg/ha,1308.30
2,Afghanistan,Production,"Almonds, in shell",2019,t,38205.00
3,Afghanistan,Area harvested,"Almonds, in shell",2020,ha,22134.00
4,Afghanistan,Yield,"Almonds, in shell",2020,kg/ha,1775.90
...,...,...,...,...,...,...
224642,Zimbabwe,Production,"Whole milk, evaporated",2019,t,5010.21
224643,Zimbabwe,Production,"Whole milk, evaporated",2020,t,4795.70
224644,Zimbabwe,Production,"Whole milk, evaporated",2021,t,4921.05
224645,Zimbabwe,Production,"Whole milk, evaporated",2022,t,4966.37


In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224647 entries, 0 to 224646
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Area     224647 non-null  object 
 1   Element  224647 non-null  object 
 2   Item     224647 non-null  object 
 3   Year     224647 non-null  int64  
 4   Unit     224645 non-null  object 
 5   Value    218130 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.3+ MB


In [13]:
df2.isnull().sum()

Area            0
Element         0
Item            0
Year            0
Unit            2
Value        6517
continent       0
dtype: int64

In [10]:
missing_count = df2.isnull().sum()
missing_percent = (missing_count/len(df2)) * 100
summary = pd.concat(
    [missing_count,missing_percent],axis=1, keys=['Count','Percent']
)
print(summary[summary['Count'] > 0].sort_values('Count', ascending=False))

       Count   Percent
Value   6517  2.900996
Unit       2  0.000890


In [11]:
# Checking for duplicate values
df2.duplicated().sum()

np.int64(1072)

In [12]:
# Duplicates values are removed
df2 = df2.drop_duplicates(keep=False)

In [13]:
# Data Transformation
# Filtering the data based on Elements and renaming the value columns
production_df = df2[df2['Element']=='Production'][['Area','Item','Year','Value']].rename(columns={'Value':'production_in_t'})
area_df = df2[df2['Element']=='Area harvested'][['Area','Item','Year','Value']].rename(columns={'Value':'area_in_h'})
yield_df = df2[df2['Element']=='Yield'][['Area','Item','Year','Value']].rename(columns={'Value':'yield_in_kg_per_h'})

In [14]:
# Merging the data based on area, item and year
df = production_df.merge(area_df,on=['Area','Item','Year'], how ='outer')
df = df.merge(yield_df,on=['Area','Item','Year'], how ='outer')

In [15]:
# Rounding off the values in the area, production and yield columns
df['area_in_h'] = df['area_in_h'].round()
df['production_in_t']=df['production_in_t'].round()
df['yield_in_kg_per_h']=df['yield_in_kg_per_h'].round()

In [ ]:
# df['area_in_h'] = df['area_in_h']/1000
# df['production_in_t'] = df['production_in_t']/1000
# df['yield_in_kg_per_h'] = df['yield_in_kg_per_h']/100

In [38]:
# Converting the Year column to datetime format
df['Year']= pd.to_datetime(df['Year'])

In [16]:
df.rename(
    columns = {'Area':'Country'}, inplace=True
)

In [52]:
df['Year'] = df['Year'].astype(str).str[-4:].astype(int)

In [17]:
# Replacing the missing values as np.nan
df.replace(["", "Missing value", "-", "null", "NA"], np.nan, inplace=True)

In [18]:
df

,Country,Item,Year,production_in_t,area_in_h,yield_in_kg_per_h
0,Afghanistan,"Almonds, in shell",2019,38205.0,29203.0,1308.0
1,Afghanistan,"Almonds, in shell",2020,39307.0,22134.0,1776.0
2,Afghanistan,"Almonds, in shell",2021,64256.0,36862.0,1743.0
3,Afghanistan,"Almonds, in shell",2022,63515.0,36462.0,1742.0
4,Afghanistan,"Almonds, in shell",2023,67000.0,37000.0,1811.0
...,...,...,...,...,...,...
89693,Zimbabwe,"Whole milk, evaporated",2019,5010.0,NaN,NaN
89694,Zimbabwe,"Whole milk, evaporated",2020,4796.0,NaN,NaN
89695,Zimbabwe,"Whole milk, evaporated",2021,4921.0,NaN,NaN
89696,Zimbabwe,"Whole milk, evaporated",2022,4966.0,NaN,NaN


In [19]:
df.to_csv('draft2.csv', index=False)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89698 entries, 0 to 89697
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Country            89698 non-null  object 
 1   Item               89698 non-null  object 
 2   Year               89698 non-null  int64  
 3   production_in_t    86181 non-null  float64
 4   area_in_h          49308 non-null  float64
 5   yield_in_kg_per_h  54888 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 4.1+ MB


In [21]:
df.isna().sum()

Country                  0
Item                     0
Year                     0
production_in_t       3517
area_in_h            40390
yield_in_kg_per_h    34810
dtype: int64

In [22]:
df.isnull().sum()

Country                  0
Item                     0
Year                     0
production_in_t       3517
area_in_h            40390
yield_in_kg_per_h    34810
dtype: int64